In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # see issue #152
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # -1 !!!!

import keras
from keras import losses
import numpy as np
from numpy import genfromtxt
from mlp_model import MLPModel
from metrics import Metrics
import csv, sys
from keras.utils import plot_model
from ann_visualizer.visualize import ann_viz;
from scipy.stats import hmean
import time


/home/sarcasm/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_dataset_verysmall = genfromtxt('train_dataset_140.csv', delimiter=',')
train_dataset_small = genfromtxt('train_dataset_1400.csv', delimiter=',')
train_dataset_medium = genfromtxt('train_dataset_14000.csv', delimiter=',')
train_dataset_large = genfromtxt('train_dataset_49000.csv', delimiter=',')
cross_validation_dataset = genfromtxt('validation_dataset2.csv', delimiter=',')
test_dataset = genfromtxt('test_dataset2.csv', delimiter=',')

# Shuffle Train Data
np.random.shuffle(train_dataset_verysmall)
np.random.shuffle(train_dataset_small)
np.random.shuffle(train_dataset_medium)
np.random.shuffle(train_dataset_large)

# Split into Inputs and Labels
train_inputs_verysmall, train_labels_verysmall = np.hsplit(train_dataset_verysmall, 2)
train_inputs_verysmall = np.array(train_inputs_verysmall).reshape(len(train_dataset_verysmall), 34)
train_labels_verysmall = np.array(train_labels_verysmall).reshape(len(train_dataset_verysmall), 34)

train_inputs_small, train_labels_small = np.hsplit(train_dataset_small, 2)
train_inputs_small = np.array(train_inputs_small).reshape(len(train_dataset_small), 34)
train_labels_small = np.array(train_labels_small).reshape(len(train_dataset_small), 34)

train_inputs_medium , train_labels_medium  = np.hsplit(train_dataset_medium, 2)
train_inputs_medium = np.array(train_inputs_medium).reshape(len(train_dataset_medium), 34)
train_labels_medium = np.array(train_labels_medium).reshape(len(train_dataset_medium), 34)

train_inputs_large, train_labels_large = np.hsplit(train_dataset_large, 2)
train_inputs_large = np.array(train_inputs_large).reshape(len(train_dataset_large), 34)
train_labels_large = np.array(train_labels_large).reshape(len(train_dataset_large), 34)

cross_validation_inputs, cross_validation_labels = np.hsplit(cross_validation_dataset, 2)
cross_validation_inputs = np.array(cross_validation_inputs).reshape(len(cross_validation_dataset), 34)
cross_validation_labels = np.array(cross_validation_labels).reshape(len(cross_validation_dataset), 34)

test_inputs, test_labels = np.hsplit(test_dataset, 2)
test_inputs = np.array(test_inputs).reshape(len(test_dataset), 34)
test_labels = np.array(test_labels).reshape(len(test_dataset), 34)

# Normalize Data
gigabit_max_throughput = 125 * 1e6 # 1000 MB/s
gigabit_min_throughput = 1e2       # 100 bytes/s
norm = gigabit_max_throughput - gigabit_min_throughput

train_inputs_verysmall = (train_inputs_verysmall - gigabit_min_throughput) / norm
train_inputs_small = (train_inputs_small - gigabit_min_throughput) / norm
train_inputs_medium = (train_inputs_medium - gigabit_min_throughput) / norm
train_inputs_large = (train_inputs_large - gigabit_min_throughput) / norm
cross_validation_inputs = (cross_validation_inputs - gigabit_min_throughput) / norm
test_inputs = (test_inputs - gigabit_min_throughput) / norm

very_small_data = (train_inputs_verysmall, train_labels_verysmall)
small_data = (train_inputs_small, train_labels_small)
medium_data = (train_inputs_medium, train_labels_medium)
large_data = (train_inputs_large, train_labels_large)

valid_routes = genfromtxt('valid_routes.csv', delimiter=',')

In [3]:
configs = []
                    
for data in [very_small_data, small_data, medium_data, large_data]:
    for loss in [losses.mean_squared_error]:
        for activation in ['linear']:
            for hu in [410]:
                for hl in [2]:
                    c = { 'activation': activation, 'data': data, 'loss': loss, 'hu': hu, 'hl': hl }
                    configs.append(c)

print("{} total configs".format(len(configs)))

4 total configs


In [4]:
table = [['loss', 'activaiton', 'data size', 'hu', 'hl', 'optimal', 'valid']]
fr = open('MLP_results_adam.csv', 'w', newline='')
writer = csv.writer(fr)

progress = 3
current_progress = -1

for config in configs:
    current_progress += 1
    if current_progress < progress:
        print('skip')
        continue
    
    start = time.time()
    model = MLPModel(
        optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-5, amsgrad=False),
        loss = config['loss'],
        hidden_units = config['hu'],
        hidden_layers = config['hl'],
        activation = config['activation'],
        name = "data{} ".format(config['data'][0].shape[0])
    )
    
    train_inputs = config['data'][0]
    train_labels = config['data'][1]

    batch = train_labels.shape[0]
    model.fit(train_inputs, train_labels, (cross_validation_inputs, cross_validation_labels), epochs = 1000, bs = 64)

    train_prediction = model.predict(train_inputs)
    train_metrcis = Metrics(train_prediction, train_labels)

    crossvalidation_prediction = model.predict(cross_validation_inputs)
    validation_metrcis = Metrics(crossvalidation_prediction, cross_validation_labels)

    test_prediction = model.predict(test_inputs)
    test_metrcis = Metrics(test_prediction, test_labels)
    
    name = "adam {} d{} hu{} hl{}".format(config['activation'], batch, config['hu'], config['hl'])
    print(name)
    print("{} Training took {}".format(current_progress, time.time() - start))
    print("Optimal predictions: {}".format(train_metrcis.optimal_predictions()))
    print("Valid predictions: {}".format(train_metrcis.valid_predictions()))
    print("H-mean: {}".format(hmean([train_metrcis.opt(), train_metrcis.corr()])))
    print("Validation")
    print("Optimal predictions: {}".format(validation_metrcis.optimal_predictions()))
    print("Valid predictions: {}".format(validation_metrcis.valid_predictions()))
    print("H-mean: {}".format(hmean([validation_metrcis.opt(), validation_metrcis.corr()])))
    print(model.evaluate(x = test_inputs, y = test_labels))
    print("==================================\n")
    
    table_row = []
    table_row.append(config['loss'])
    table_row.append(config['activation'])
    table_row.append(batch)
    table_row.append(config['hu'])
    table_row.append(config['hl'])
    table_row.append(test_metrcis.optimal_count)
    table_row.append(test_metrcis.valid_count)
    table.append(table_row)
    
    directory = "MLP/{}".format(name)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    f = open(directory + "/predictions", 'w')
    for pi in range(len(test_prediction)):
        f.write(str(test_prediction[pi]))
        f.write("\n")
        f.write(str(test_labels[pi]))
        f.write("\n")
        f.write("\n")
    f.close
    
    writer.writerow(table_row)


    fr.close

skip
skip
skip
adam linear d49000 hu410 hl2
3 Training took 6112.2840740680695
Optimal predictions: 0.20204081632653062 (9900/49000)
Valid predictions: 0.25265306122448977 (12380/49000)
H-mean: 0.224530099292859
Validation
Optimal predictions: 0.21285714285714286 (149/700)
Valid predictions: 0.24857142857142858 (174/700)
H-mean: 0.22933215391419728
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 65us/step
[0.10534382494432586, 0.08571428571428572, 0.8573949411937168, 0.10534382494432586, 0.3309360147374017]



In [5]:
Ex1
adam sigmoidd1400 hu62 hl2
0 Training took 71.8241834640503
Optimal predictions: 0.7714285714285715 (1080/1400)
Valid predictions: 0.8142857142857143 (1140/1400)
Validation
Optimal predictions: 0.5457142857142857 (382/700)
Valid predictions: 0.7142857142857143 (500/700)
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 54us/step
[0.07967027817453656, 0.0014285714285714286, 0.8953781454903739, 0.07967027817453656, 0.29157771199941634]
==================================

adam sigmoidd1400 hu410 hl2
1 Training took 184.9844319820404
Optimal predictions: 0.98 (1372/1400)
Valid predictions: 0.985 (1379/1400)
Validation
Optimal predictions: 0.52 (364/700)
Valid predictions: 0.6985714285714286 (489/700)
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 72us/step
[0.09294128000736236, 0.07428571428571429, 0.8869747761317662, 0.09294128000736236, 0.4651767077616283]
==================================

adam sigmoidd1400 hu760 hl2
2 Training took 579.9031939506531
Optimal predictions: 0.985 (1379/1400)
Valid predictions: 0.9892857142857143 (1385/1400)
Validation
Optimal predictions: 0.5114285714285715 (358/700)
Valid predictions: 0.6857142857142857 (480/700)
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 113us/step
[0.0962618936598301, 0.10428571441343852, 0.8867647079059056, 0.0962618936598301, 0.5038616906745094]
==================================

adam sigmoidd1400 hu1110 hl2
3 Training took 1382.5484766960144
Optimal predictions: 0.985 (1379/1400)
Valid predictions: 0.9885714285714285 (1384/1400)
Validation
Optimal predictions: 0.5228571428571429 (366/700)
Valid predictions: 0.7042857142857143 (493/700)
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 138us/step
[0.09737703421286174, 0.07000000004257474, 0.8851260566711425, 0.09737703421286174, 0.4961001022372927]
==================================
adam sigmoidd1400 hu1456 hl2
4 Training took 2366.626567840576
Optimal predictions: 0.9878571428571429 (1383/1400)
Valid predictions: 0.9921428571428571 (1389/1400)
Validation
Optimal predictions: 0.5071428571428571 (355/700)
Valid predictions: 0.7085714285714285 (496/700)
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 311us/step
[0.09764283152563231, 0.06285714285714286, 0.8847479023252215, 0.09764283152563231, 0.49519609459808894]
==================================

SyntaxError: invalid syntax (<ipython-input-5-c2b275a0aad0>, line 2)

In [ ]:
Ex2
adam lineard1400 hu410 hl2
0 Training took 209.9992482662201
Optimal predictions: 0.2 (280/1400)
Valid predictions: 0.24142857142857144 (338/1400)
H-mean: 0.21877022653721687
Validation
Optimal predictions: 0.18857142857142858 (132/700)
Valid predictions: 0.24142857142857144 (169/700)
H-mean: 0.21175130517323207
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 66us/step
[0.10811390770333154, 0.07571428571428572, 0.856932771205902, 0.10811390770333154, 0.3466015271629606]
==================================

adam sigmoidd1400 hu410 hl2
1 Training took 208.36319422721863
Optimal predictions: 0.98 (1372/1400)
Valid predictions: 0.985 (1379/1400)
H-mean: 0.9824936386768447
Validation
Optimal predictions: 0.5271428571428571 (369/700)
Valid predictions: 0.7042857142857143 (493/700)
H-mean: 0.6029731521378854
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 71us/step
[0.09732528307608196, 0.06857142857142857, 0.8840756188120161, 0.09732528307608196, 0.5100316688418388]
==================================

adam tanhd1400 hu410 hl2
2 Training took 208.184739112854
Optimal predictions: 0.9992857142857143 (1399/1400)
Valid predictions: 0.9992857142857143 (1399/1400)
H-mean: 0.9992857142857142
Validation
Optimal predictions: 0.06142857142857143 (43/700)
Valid predictions: 0.06142857142857143 (43/700)
H-mean: 0.06142857142857143
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 72us/step
[0.15694988263504847, 0.0, 0.7829831910133361, 0.15694988263504847, 0.4844191025836127]
==================================

adam softplusd1400 hu410 hl2
3 Training took 215.0997188091278
Optimal predictions: 0.9907142857142858 (1387/1400)
Valid predictions: 0.9907142857142858 (1387/1400)
H-mean: 0.9907142857142858
Validation
Optimal predictions: 0.2957142857142857 (207/700)
Valid predictions: 0.35285714285714287 (247/700)
H-mean: 0.32176840780365007
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 78us/step
[0.13870253411786898, 0.0828571428997176, 0.8515966398375375, 0.13870253411786898, 0.45543013232094903]
==================================

adam relud1400 hu410 hl2
4 Training took 189.67839121818542
Optimal predictions: 0.2857142857142857 (400/1400)
Valid predictions: 0.2857142857142857 (400/1400)
H-mean: 0.2857142857142857
Validation
Optimal predictions: 0.1 (70/700)
Valid predictions: 0.12857142857142856 (90/700)
H-mean: 0.11249999999999999
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 72us/step
[0.10453869249139514, 0.06857142863529068, 0.8716806476456779, 0.10453869249139514, 0.8848804603304182]
==================================
adam sigmoid + linear 1400 hu410 hl2
0 Training took 180.50601387023926
Optimal predictions: 0.7057142857142857 (988/1400)
Valid predictions: 0.7214285714285714 (1010/1400)
H-mean: 0.7134849134849135
Validation
Optimal predictions: 0.41714285714285715 (292/700)
Valid predictions: 0.4942857142857143 (346/700)
H-mean: 0.452449619346171
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 66us/step
[0.09459398439952306, 0.07857142857142857, 0.8815125857080732, 0.09459398439952306, 0.3455440412248884]
==================================
adam leaky-relu 1400 hu410 hl2
0 Training took 209.76030564308167
Optimal predictions: 1.0 (1400/1400)
Valid predictions: 1.0 (1400/1400)
H-mean: 1.0
Validation
Optimal predictions: 0.47714285714285715 (334/700)
Valid predictions: 0.6142857142857143 (430/700)
H-mean: 0.5370979805534779
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 65us/step
[0.08658495204789297, 0.0742857143495764, 0.8887815056528364, 0.08658495204789297, 0.31994876946721756]
==================================


In [ ]:
Ex3
adam tanhd140 hu410 hl2
0 Training took 59.5466034412384
Optimal predictions: 1.0 (140/140)
Valid predictions: 1.0 (140/140)
H-mean: 1.0
Validation
Optimal predictions: 0.03142857142857143 (22/700)
Valid predictions: 0.04857142857142857 (34/700)
H-mean: 0.03816326530612245
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 78us/step
[0.19851449012756348, 0.007142857142857143, 0.7426890689986093, 0.19851449012756348, 0.7194668180601937]
==================================

adam softplusd140 hu410 hl2
1 Training took 54.440468311309814
Optimal predictions: 0.7785714285714286 (109/140)
Valid predictions: 0.7785714285714286 (109/140)
H-mean: 0.7785714285714286
Validation
Optimal predictions: 0.09857142857142857 (69/700)
Valid predictions: 0.17714285714285713 (124/700)
H-mean: 0.12666173205033307
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 76us/step
[0.170137414761952, 0.13714285727058137, 0.8013445343290057, 0.170137414761952, 0.5710914894512722]
==================================

adam tanhd1400 hu410 hl2
2 Training took 206.7653090953827
Optimal predictions: 0.9992857142857143 (1399/1400)
Valid predictions: 0.9992857142857143 (1399/1400)
H-mean: 0.9992857142857142
Validation
Optimal predictions: 0.04 (28/700)
Valid predictions: 0.045714285714285714 (32/700)
H-mean: 0.042666666666666665
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 75us/step
[0.15696728548833302, 0.0, 0.7838235221590315, 0.15696728548833302, 0.49466174874986923]
==================================

adam softplusd1400 hu410 hl2
3 Training took 217.61182165145874
Optimal predictions: 0.9885714285714285 (1384/1400)
Valid predictions: 0.9885714285714285 (1384/1400)
H-mean: 0.9885714285714284
Validation
Optimal predictions: 0.2885714285714286 (202/700)
Valid predictions: 0.3342857142857143 (234/700)
H-mean: 0.30975098296199216
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 76us/step
[0.13318502345255442, 0.09428571428571429, 0.8511344589505877, 0.13318502345255442, 0.40443365148135596]
==================================

adam tanhd14000 hu410 hl2
4 Training took 1686.0229189395905
Optimal predictions: 0.9892142857142857 (13849/14000)
Valid predictions: 0.9896428571428572 (13855/14000)
H-mean: 0.9894285250195949
Validation
Optimal predictions: 0.5714285714285714 (400/700)
Valid predictions: 0.6385714285714286 (447/700)
H-mean: 0.6031371226176421
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 81us/step
[0.06720684072801045, 0.0, 0.9133613337789263, 0.06720684072801045, 0.2234827491215297]
==================================
softplus14000
5 Training took 1696.74786901474
Optimal predictions: 0.9938571428571429 (13914/14000)
Valid predictions: 0.994 (13916/14000)
H-mean: 0.9939285662953649
Validation
Optimal predictions: 0.6157142857142858 (431/700)
Valid predictions: 0.6671428571428571 (467/700)
H-mean: 0.6403977091950366
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 74us/step
[0.057183577671114885, 0.07142857155629567, 0.9264285799435207, 0.057183577671114885, 0.20351383197520462]
==================================

adam tanhd49000 hu410 hl2
6 Training took 5527.457977294922
Optimal predictions: 0.9639795918367347 (47235/49000)
Valid predictions: 0.966 (47334/49000)
H-mean: 0.9649887383814991
Validation
Optimal predictions: 0.8014285714285714 (561/700)
Valid predictions: 0.8414285714285714 (589/700)
H-mean: 0.8209416149068322
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 72us/step
[0.02798395073307412, 0.0014285714285714286, 0.9650000064713614, 0.02798395073307412, 0.10979120558926038]
==================================

adam softplusd49000 hu410 hl2
7 Training took 6047.1450452804565
Optimal predictions: 0.9950204081632653 (48756/49000)
Valid predictions: 0.9950816326530613 (48759/49000)
H-mean: 0.995051019466393
Validation
Optimal predictions: 0.7671428571428571 (537/700)
Valid predictions: 0.8028571428571428 (562/700)
H-mean: 0.7845937865592096
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 75us/step
[0.0332288857682475, 0.12, 0.9576050424575806, 0.0332288857682475, 0.1370690682743277]

adam leaky relud140 hu410 hl2
0 Training took 54.94974231719971
Optimal predictions: 1.0 (140/140)
Valid predictions: 1.0 (140/140)
H-mean: 1.0
Validation
Optimal predictions: 0.15714285714285714 (110/700)
Valid predictions: 0.28 (196/700)
H-mean: 0.20130718954248364
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 73us/step
[0.13455852329730988, 0.06571428571428571, 0.8147058752604893, 0.13455852329730988, 0.530158679996218]
==================================

adam leaky relud1400 hu410 hl2
1 Training took 192.6991012096405
Optimal predictions: 1.0 (1400/1400)
Valid predictions: 1.0 (1400/1400)
H-mean: 1.0
Validation
Optimal predictions: 0.4642857142857143 (325/700)
Valid predictions: 0.61 (427/700)
H-mean: 0.5272606382978724
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 68us/step
[0.0880118389427662, 0.05285714285714286, 0.8828151219231741, 0.0880118389427662, 0.3125918860094888]
==================================

adam leaky relud14000 hu410 hl2
2 Training took 1595.2359194755554
Optimal predictions: 1.0 (14000/14000)
Valid predictions: 1.0 (14000/14000)
H-mean: 1.0
Validation
Optimal predictions: 0.7485714285714286 (524/700)
Valid predictions: 0.8371428571428572 (586/700)
H-mean: 0.7903835263835264
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 70us/step
[0.042861226978046556, 0.055714285714285716, 0.9474789830616542, 0.042861226978046556, 0.16481288929070745]
==================================

adam leaky relud49000 hu410 hl2
3 Training took 5569.441777706146
Optimal predictions: 0.9999183673469387 (48996/49000)
Valid predictions: 0.9999183673469387 (48996/49000)
H-mean: 0.9999183673469387
Validation
Optimal predictions: 0.8342857142857143 (584/700)
Valid predictions: 0.8957142857142857 (627/700)
H-mean: 0.8639094019110535
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 76us/step
[0.028758717362901993, 0.10142857142857142, 0.9691596531867981, 0.028758717362901993, 0.11732415357870715]
==================================

adam relu d140 hu410 hl2
0 Training took 57.661373138427734
Optimal predictions: 1.0 (140/140)
Valid predictions: 1.0 (140/140)
H-mean: 1.0
Validation
Optimal predictions: 0.16142857142857142 (113/700)
Valid predictions: 0.25142857142857145 (176/700)
H-mean: 0.19661888284725657
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 70us/step
[0.1320107826590538, 0.07857142857142857, 0.8168067165783474, 0.1320107826590538, 0.5110279311452594]
==================================

adam sigmoid d140 hu410 hl2
1 Training took 54.20580053329468
Optimal predictions: 0.5857142857142857 (82/140)
Valid predictions: 0.6142857142857143 (86/140)
H-mean: 0.5996598639455782
Validation
Optimal predictions: 0.17714285714285713 (124/700)
Valid predictions: 0.29 (203/700)
H-mean: 0.2199388379204893
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 71us/step
[0.1321660402417183, 0.06285714285714286, 0.8266386488505773, 0.1321660402417183, 0.5677690517902374]
==================================

adam relu d1400 hu410 hl2
2 Training took 208.93197989463806
Optimal predictions: 1.0 (1400/1400)
Valid predictions: 1.0 (1400/1400)
H-mean: 1.0
Validation
Optimal predictions: 0.4642857142857143 (325/700)
Valid predictions: 0.6171428571428571 (432/700)
H-mean: 0.5299113040196263
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 68us/step
[0.08665535603250776, 0.0642857143495764, 0.8851260478155953, 0.08665535603250776, 0.3086826809814998]
==================================

adam sigmoid d1400 hu410 hl2
3 Training took 210.22500467300415
Optimal predictions: 0.9942857142857143 (1392/1400)
Valid predictions: 0.9942857142857143 (1392/1400)
H-mean: 0.9942857142857142
Validation
Optimal predictions: 0.43142857142857144 (302/700)
Valid predictions: 0.5842857142857143 (409/700)
H-mean: 0.49635523407675314
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 77us/step
[0.10786061455096517, 0.09428571441343853, 0.8678151147706168, 0.10786061455096517, 0.49914869074310575]
==================================

adam relu d14000 hu410 hl2
4 Training took 1699.563291311264
Optimal predictions: 1.0 (14000/14000)
Valid predictions: 1.0 (14000/14000)
H-mean: 1.0
Validation
Optimal predictions: 0.7614285714285715 (533/700)
Valid predictions: 0.8557142857142858 (599/700)
H-mean: 0.8058228167592125
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 75us/step
[0.04068276911707861, 0.10857142908232553, 0.9468907567432948, 0.04068276911707861, 0.15831540500479085]
==================================

adam sigmoid d14000 hu410 hl2
5 Training took 2049.152975320816
Optimal predictions: 0.9999285714285714 (13999/14000)
Valid predictions: 0.9999285714285714 (13999/14000)
H-mean: 0.9999285714285714
Validation
Optimal predictions: 0.6628571428571428 (464/700)
Valid predictions: 0.7842857142857143 (549/700)
H-mean: 0.7184769426032999
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 81us/step
[0.0753557847706335, 0.09142857155629566, 0.9073949599266052, 0.0753557847706335, 0.3273747247883252]
==================================

adam relu d49000 hu410 hl2
6 Training took 5603.25888466835
Optimal predictions: 0.9996938775510205 (48985/49000)
Valid predictions: 0.9997959183673469 (48990/49000)
H-mean: 0.9997448953554374
Validation
Optimal predictions: 0.8485714285714285 (594/700)
Valid predictions: 0.9071428571428571 (635/700)
H-mean: 0.8768801580843891
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 70us/step
[0.03236284171231091, 0.09000000012772424, 0.9594117617607116, 0.03236284171231091, 0.13595326741891248]
==================================

adam sigmoid d49000 hu410 hl2
7 Training took 5466.491918325424
Optimal predictions: 0.9962857142857143 (48818/49000)
Valid predictions: 0.9972040816326531 (48863/49000)
H-mean: 0.9967446864209593
Validation
Optimal predictions: 0.8128571428571428 (569/700)
Valid predictions: 0.8628571428571429 (604/700)
H-mean: 0.8371111923030082
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 77us/step
[0.038850140066019126, 0.09000000006386212, 0.9540336128643581, 0.038850140066019126, 0.14530422284134797]
==================================

adam linear d140 hu410 hl2
0 Training took 50.66955924034119
Optimal predictions: 0.2857142857142857 (40/140)
Valid predictions: 0.32142857142857145 (45/140)
H-mean: 0.3025210084033614
Validation
Optimal predictions: 0.07571428571428572 (53/700)
Valid predictions: 0.11571428571428571 (81/700)
H-mean: 0.09153518123667378
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 73us/step
[0.14359479750905718, 0.10142857142857142, 0.8018487405776977, 0.14359479750905718, 0.5394926866463252]
==================================

adam linear d1400 hu410 hl2
1 Training took 178.5749626159668
Optimal predictions: 0.19428571428571428 (272/1400)
Valid predictions: 0.2357142857142857 (330/1400)
H-mean: 0.21300427147603224
Validation
Optimal predictions: 0.19857142857142857 (139/700)
Valid predictions: 0.24428571428571427 (171/700)
H-mean: 0.21906912442396315
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 68us/step
[0.10822141585605485, 0.07571428571428572, 0.8562184797014509, 0.10822141585605485, 0.3427330957991736]
==================================

adam linear d14000 hu410 hl2
2 Training took 1520.4463455677032
Optimal predictions: 0.19807142857142856 (2773/14000)
Valid predictions: 0.24585714285714286 (3442/14000)
H-mean: 0.21939239167911737
Validation
Optimal predictions: 0.21 (147/700)
Valid predictions: 0.25 (175/700)
H-mean: 0.22826086956521738
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 71us/step
[0.10577943569847516, 0.07142857142857142, 0.8573109170368739, 0.10577943569847516, 0.33650412363665444]
==================================
adam linear d49000 hu410 hl2
3 Training took 6112.2840740680695
Optimal predictions: 0.20204081632653062 (9900/49000)
Valid predictions: 0.25265306122448977 (12380/49000)
H-mean: 0.224530099292859
Validation
Optimal predictions: 0.21285714285714286 (149/700)
Valid predictions: 0.24857142857142858 (174/700)
H-mean: 0.22933215391419728
['loss', 'acc', 'binary_accuracy', 'mean_squared_error', 'binary_crossentropy']
700/700 [==============================] - 0s 65us/step
[0.10534382494432586, 0.08571428571428572, 0.8573949411937168, 0.10534382494432586, 0.3309360147374017]
==================================